In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)

In [3]:
spark = SparkSession \
    .builder \
    .appName("CSV with conditions") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

#????????? não percebi bem se o config é preciso em alguns não aparece

In [4]:
df = spark.read.csv("dataset/conditions.csv") .limit(50)

In [5]:
df.printSchema()
df.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+----------+----------+--------------------+--------------------+---------+--------------------+
|       _c0|       _c1|                 _c2|                 _c3|      _c4|                 _c5|
+----------+----------+--------------------+--------------------+---------+--------------------+
|     START|      STOP|             PATIENT|           ENCOUNTER|     CODE|         DESCRIPTION|
|2017-01-14|2017-03-30|09e4e8cb-29c2-4ef...|88e540ab-a7d7-47d...| 65363002|        Otitis media|
|2012-09-15|2012-09-16|b0a03e8c-8d0f-424...|e89414dc-d0c6-478...|241929008|Acute allergic re...|
|2018-06-17|2018-06-24|09e4e8cb-29c2-4ef...|c14325b0-f7ec-431...|444814009|Viral sinusitis (...|
|2019-04-19|2019-09-26|09e4e8cb-29c2-4ef...|71af18ee-3157-408...| 65363002|        Otitis media|
|2019-04

In [6]:
Data_list = ["START","STOP","PATIENT","ENCOUNTER","CODE","DESCRIPTION"]
 
df = df.toDF(*Data_list)
df=df.filter(df.START !="START")
df=df.drop(*['START','STOP','ENCOUNTER','DESCRIPTION'])
df.show()

+--------------------+---------+
|             PATIENT|     CODE|
+--------------------+---------+
|09e4e8cb-29c2-4ef...| 65363002|
|b0a03e8c-8d0f-424...|241929008|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 65363002|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 33737001|
|b0a03e8c-8d0f-424...|444814009|
|b0a03e8c-8d0f-424...| 10509002|
|b0a03e8c-8d0f-424...|233678006|
|b0a03e8c-8d0f-424...|195662009|
|b0a03e8c-8d0f-424...|232353008|
|b0a03e8c-8d0f-424...|195662009|
|5420ae87-24c8-4ed...|446096008|
|5420ae87-24c8-4ed...|284551006|
|5420ae87-24c8-4ed...|283371005|
|5420ae87-24c8-4ed...| 72892002|
|5420ae87-24c8-4ed...|444814009|
|5420ae87-24c8-4ed...|195662009|
|bf1f30f2-27de-4b5...|162864005|
|bf1f30f2-27de-4b5...|283385000|
+--------------------+---------+
only showing top 20 rows



In [7]:
df = df.dropna()  #confirmar depois com tudo

In [8]:
df.show()

+--------------------+---------+
|             PATIENT|     CODE|
+--------------------+---------+
|09e4e8cb-29c2-4ef...| 65363002|
|b0a03e8c-8d0f-424...|241929008|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 65363002|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 33737001|
|b0a03e8c-8d0f-424...|444814009|
|b0a03e8c-8d0f-424...| 10509002|
|b0a03e8c-8d0f-424...|233678006|
|b0a03e8c-8d0f-424...|195662009|
|b0a03e8c-8d0f-424...|232353008|
|b0a03e8c-8d0f-424...|195662009|
|5420ae87-24c8-4ed...|446096008|
|5420ae87-24c8-4ed...|284551006|
|5420ae87-24c8-4ed...|283371005|
|5420ae87-24c8-4ed...| 72892002|
|5420ae87-24c8-4ed...|444814009|
|5420ae87-24c8-4ed...|195662009|
|bf1f30f2-27de-4b5...|162864005|
|bf1f30f2-27de-4b5...|283385000|
+--------------------+---------+
only showing top 20 rows



In [9]:
#Cada basquet não tem items repetidos, ou seja, se uma pessoa tiver a mesma doença mais que uma vez não é contabilizado aqui
#Pode ser interessante arranjar maneira de contabilizar, logo se vê....

from pyspark.sql.functions import collect_set, col, count

baskets = df.groupBy('PATIENT').agg(collect_set('CODE').alias('items'))
baskets.createOrReplaceTempView('baskets')
baskets.show(20,False)

+------------------------------------+------------------------------------------------------------------------------------------------+
|PATIENT                             |items                                                                                           |
+------------------------------------+------------------------------------------------------------------------------------------------+
|09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a|[33737001, 65363002, 444814009]                                                                 |
|b0a03e8c-8d0f-4242-9548-40f4d294eba8|[241929008, 10509002, 233678006, 444814009, 195662009, 232353008]                               |
|5420ae87-24c8-4ed4-ad14-041d15aadae9|[446096008, 72892002, 283371005, 444814009, 195662009, 284551006]                               |
|bf1f30f2-27de-4b54-809b-f91b92949565|[162864005, 283385000, 239873007]                                                               |
|28a3cdb7-1db1-4148-8280-8a4e5b4f99e0|[156073000

In [10]:
baskets=baskets.drop('PATIENT')
baskets.show(20,False)

+------------------------------------------------------------------------------------------------+
|items                                                                                           |
+------------------------------------------------------------------------------------------------+
|[33737001, 65363002, 444814009]                                                                 |
|[241929008, 10509002, 233678006, 444814009, 195662009, 232353008]                               |
|[446096008, 72892002, 283371005, 444814009, 195662009, 284551006]                               |
|[162864005, 283385000, 239873007]                                                               |
|[156073000, 72892002, 19169002, 284551006]                                                      |
|[162864005, 713197008, 429007001, 36971009, 444814009, 410429000, 55822004, 239873007, 68496003]|
|[307731004, 72892002, 10509002, 444814009, 195662009, 44465007]                                 |
|[16286400

In [11]:
BasquetRDD = baskets.rdd.flatMap(list)   #.map(list)
BasquetRDD.take(5)

[['33737001', '65363002', '444814009'],
 ['241929008', '10509002', '233678006', '444814009', '195662009', '232353008'],
 ['446096008', '72892002', '283371005', '444814009', '195662009', '284551006'],
 ['162864005', '283385000', '239873007'],
 ['156073000', '72892002', '19169002', '284551006']]

In [12]:
dfCodes=df.drop('PATIENT')
dfCodes.show(20,False)

+---------+
|CODE     |
+---------+
|65363002 |
|241929008|
|444814009|
|65363002 |
|444814009|
|33737001 |
|444814009|
|10509002 |
|233678006|
|195662009|
|232353008|
|195662009|
|446096008|
|284551006|
|283371005|
|72892002 |
|444814009|
|195662009|
|162864005|
|283385000|
+---------+
only showing top 20 rows



In [13]:
CodesRDD=dfCodes.rdd.flatMap(list)
CodesRDD.take(20)

['65363002',
 '241929008',
 '444814009',
 '65363002',
 '444814009',
 '33737001',
 '444814009',
 '10509002',
 '233678006',
 '195662009',
 '232353008',
 '195662009',
 '446096008',
 '284551006',
 '283371005',
 '72892002',
 '444814009',
 '195662009',
 '162864005',
 '283385000']

In [14]:
Unique = CodesRDD.distinct()
Unique.take(10)

['65363002',
 '241929008',
 '444814009',
 '33737001',
 '10509002',
 '233678006',
 '195662009',
 '232353008',
 '446096008',
 '284551006']

In [15]:
CountRDD = CodesRDD.map(lambda item: (item, 1))
CountRDD = CountRDD.reduceByKey(lambda a,b: a+b)
CountRDD.take(10)

[('65363002', 2),
 ('241929008', 1),
 ('444814009', 6),
 ('33737001', 1),
 ('10509002', 3),
 ('233678006', 1),
 ('195662009', 4),
 ('232353008', 1),
 ('446096008', 1),
 ('284551006', 2)]

In [16]:
#Counts = CountRDD.map(lambda x: x[1])
#Counts.take(10)
# esta parte acho que não precisamos

In [34]:
# Setting a support threshold of 1000
# Aqui não tá 1000, tá 2, mas na versão com tudo faz-se com 1000
CountFilterRDD=CountRDD.filter(lambda x: x[1] >= 2) #Aqui 1000
CountFilterRDD.take(10)

[('65363002', 2),
 ('444814009', 6),
 ('10509002', 3),
 ('195662009', 4),
 ('284551006', 2),
 ('72892002', 7),
 ('162864005', 3),
 ('239873007', 2),
 ('55822004', 2)]

In [35]:
BaseRDD=CountFilterRDD.map(lambda x: ([x[0]], x[1]))
BaseRDD.take(10)

[(['65363002'], 2),
 (['444814009'], 6),
 (['10509002'], 3),
 (['195662009'], 4),
 (['284551006'], 2),
 (['72892002'], 7),
 (['162864005'], 3),
 (['239873007'], 2),
 (['55822004'], 2)]

In [36]:
CodeFilterRDD=CountFilterRDD.map(lambda x: x[0])
CodeFilterRDD.take(10)

['65363002',
 '444814009',
 '10509002',
 '195662009',
 '284551006',
 '72892002',
 '162864005',
 '239873007',
 '55822004']

In [23]:
#Simplesmente copiei isto temos de perceber e mudar ahah

# Remove Replicas 
#Basquets with items that are in a diferent order, but with the same items

def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [25]:
Combine = CodeFilterRDD.cartesian(Unique)
Combine.take(40)

[('65363002', '65363002'),
 ('65363002', '241929008'),
 ('65363002', '444814009'),
 ('65363002', '33737001'),
 ('65363002', '10509002'),
 ('65363002', '233678006'),
 ('65363002', '195662009'),
 ('65363002', '232353008'),
 ('65363002', '446096008'),
 ('65363002', '284551006'),
 ('65363002', '283371005'),
 ('65363002', '72892002'),
 ('65363002', '162864005'),
 ('65363002', '283385000'),
 ('65363002', '239873007'),
 ('65363002', '19169002'),
 ('65363002', '156073000'),
 ('65363002', '410429000'),
 ('65363002', '429007001'),
 ('65363002', '55822004'),
 ('65363002', '307731004'),
 ('65363002', '68496003'),
 ('65363002', '44465007'),
 ('65363002', '713197008'),
 ('65363002', '36971009'),
 ('65363002', '24079001'),
 ('65363002', '65966004'),
 ('444814009', '65363002'),
 ('10509002', '65363002'),
 ('444814009', '241929008'),
 ('444814009', '444814009'),
 ('10509002', '241929008'),
 ('10509002', '444814009'),
 ('444814009', '33737001'),
 ('444814009', '10509002'),
 ('444814009', '233678006'),
 

In [26]:
Combine = Combine.map(lambda item: removeReplica(item))
Combine.take(40)

[['65363002'],
 ('241929008', '65363002'),
 ('444814009', '65363002'),
 ('33737001', '65363002'),
 ('10509002', '65363002'),
 ('233678006', '65363002'),
 ('195662009', '65363002'),
 ('232353008', '65363002'),
 ('446096008', '65363002'),
 ('284551006', '65363002'),
 ('283371005', '65363002'),
 ('65363002', '72892002'),
 ('162864005', '65363002'),
 ('283385000', '65363002'),
 ('239873007', '65363002'),
 ('19169002', '65363002'),
 ('156073000', '65363002'),
 ('410429000', '65363002'),
 ('429007001', '65363002'),
 ('55822004', '65363002'),
 ('307731004', '65363002'),
 ('65363002', '68496003'),
 ('44465007', '65363002'),
 ('65363002', '713197008'),
 ('36971009', '65363002'),
 ('24079001', '65363002'),
 ('65363002', '65966004'),
 ('444814009', '65363002'),
 ('10509002', '65363002'),
 ('241929008', '444814009'),
 ['444814009'],
 ('10509002', '241929008'),
 ('10509002', '444814009'),
 ('33737001', '444814009'),
 ('10509002', '444814009'),
 ('233678006', '444814009'),
 ('195662009', '444814009'

In [27]:
def SumOp(x,y):
    return x+y

In [39]:
# assim acho que só funciona para k=2
# ainda tem de se ver melhor

c=2
#for c in[2,3]:
#while(CodeFilterRDD.isEmpty() == False):

combined = CodeFilterRDD.cartesian(Unique)
combined = combined.map(lambda item: removeReplica(item))
  
combined = combined.filter(lambda item: len(item) == c)
combined = combined.distinct()

    
combined_2 = combined.cartesian(BasquetRDD)
combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
combined_2 = combined_2.map(lambda item: item[0])
combined_2 = combined_2.map(lambda item: (item , 1))
combined_2 = combined_2.reduceByKey(SumOp)
combined_2 = combined_2.filter(lambda item: item[1] >= 2) #Aqui 1000 em vez de 2

BaseRDD = BaseRDD.union(combined_2)
    
combined_2 = combined_2.map(lambda item: item[0])
supportRdd = combined_2
print(c ,'. Table has crated... ')

#c = c+1 

2 . Table has crated... 


In [40]:
combined.take(5)

[('241929008', '65363002'),
 ('444814009', '65363002'),
 ('33737001', '65363002'),
 ('10509002', '65363002'),
 ('233678006', '65363002')]

In [41]:
CodeFilterRDD.take(10)

['65363002',
 '444814009',
 '10509002',
 '195662009',
 '284551006',
 '72892002',
 '162864005',
 '239873007',
 '55822004']

In [42]:
BasquetRDD.take(10)

[['33737001', '65363002', '444814009'],
 ['241929008', '10509002', '233678006', '444814009', '195662009', '232353008'],
 ['446096008', '72892002', '283371005', '444814009', '195662009', '284551006'],
 ['162864005', '283385000', '239873007'],
 ['156073000', '72892002', '19169002', '284551006'],
 ['162864005',
  '713197008',
  '429007001',
  '36971009',
  '444814009',
  '410429000',
  '55822004',
  '239873007',
  '68496003'],
 ['307731004', '72892002', '10509002', '444814009', '195662009', '44465007'],
 ['162864005', '65966004', '24079001', '55822004']]

In [44]:
BaseRDD.take(15)

[(['65363002'], 2),
 (['444814009'], 6),
 (['10509002'], 3),
 (['195662009'], 4),
 (['284551006'], 2),
 (['72892002'], 7),
 (['162864005'], 3),
 (['239873007'], 2),
 (['55822004'], 2),
 (('10509002', '444814009'), 2),
 (('195662009', '444814009'), 3),
 (('10509002', '195662009'), 2),
 (('444814009', '72892002'), 2),
 (('195662009', '72892002'), 2),
 (('284551006', '72892002'), 2)]

In [45]:
combined_2.take(10)

[('10509002', '444814009'),
 ('195662009', '444814009'),
 ('10509002', '195662009'),
 ('444814009', '72892002'),
 ('195662009', '72892002'),
 ('284551006', '72892002'),
 ('162864005', '239873007'),
 ('162864005', '55822004')]

In [ ]:
### Confidence Calculation
# Nem sei bem o que é ainda

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r %s %s |%s| %s%% %s' % ('Preprocessing ' , prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


In [ ]:
# falta ainda uma parte do apriori

Repositório fixe para o apriori

https://github.com/sergencansiz/apriori-pyspark/blob/master/AprioriPySpark.py
